In [ ]:
import json
import urllib.parse
import boto3
from datetime import datetime

s3 = boto3.client('s3')
rekognition = boto3.client('rekognition')

def lambda_handler(event, context):
    print("Received event: " + json.dumps(event))
    bucket = event['Records'][0]['s3']['bucket']['name']
    key = urllib.parse.unquote_plus(event['Records'][0]['s3']['object']['key'], encoding='utf-8')
    today = datetime.now()
    try:
        response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
            MaxLabels=10)
        labels = [x['Name'] for x in response['Labels']]
        print("detected labels : {}".format(labels))
        return {
            "objectKey": key,
            "bucket": bucket,
            'createdTimestamp': today.isoformat()
            "labels": labels
        }
    except Exception as e:
        print(e)
        print('Error getting object {} from bucket {}.'.format(key, bucket))
        raise e


In [4]:
import json
import urllib.parse
import boto3
from datetime import datetime

s3 = boto3.client('s3')
response = s3.head_object(Bucket='bucket-album', Key='se.jpg')

print('Response: {}'.format(response))

Response: {'ResponseMetadata': {'RequestId': '9SYW3RMCM9S98GPX', 'HostId': 'XU7JgM0wRsnmfuOygZ2Y9NIrL0X8PQFzh0gLpqZyrKBu6PhoPCmfzg+vClxDXcgg5s9MGjWcK28=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': 'XU7JgM0wRsnmfuOygZ2Y9NIrL0X8PQFzh0gLpqZyrKBu6PhoPCmfzg+vClxDXcgg5s9MGjWcK28=', 'x-amz-request-id': '9SYW3RMCM9S98GPX', 'date': 'Mon, 07 Nov 2022 01:12:31 GMT', 'last-modified': 'Mon, 07 Nov 2022 01:07:59 GMT', 'etag': '"b1b35b0572cbf3d4a0c8fa08f361ae0a"', 'x-amz-meta-customlabels': 'hello,world', 'accept-ranges': 'bytes', 'content-type': 'image/jpeg', 'server': 'AmazonS3', 'content-length': '103809'}, 'RetryAttempts': 0}, 'AcceptRanges': 'bytes', 'LastModified': datetime.datetime(2022, 11, 7, 1, 7, 59, tzinfo=tzutc()), 'ContentLength': 103809, 'ETag': '"b1b35b0572cbf3d4a0c8fa08f361ae0a"', 'ContentType': 'image/jpeg', 'Metadata': {'customlabels': 'hello,world'}}


In [6]:
response['ResponseMetadata']['HTTPHeaders']

{'x-amz-id-2': 'XU7JgM0wRsnmfuOygZ2Y9NIrL0X8PQFzh0gLpqZyrKBu6PhoPCmfzg+vClxDXcgg5s9MGjWcK28=',
 'x-amz-request-id': '9SYW3RMCM9S98GPX',
 'date': 'Mon, 07 Nov 2022 01:12:31 GMT',
 'last-modified': 'Mon, 07 Nov 2022 01:07:59 GMT',
 'etag': '"b1b35b0572cbf3d4a0c8fa08f361ae0a"',
 'x-amz-meta-customlabels': 'hello,world',
 'accept-ranges': 'bytes',
 'content-type': 'image/jpeg',
 'server': 'AmazonS3',
 'content-length': '103809'}

In [1]:
import boto3

def detect_labels(photo, bucket):
    client=boto3.client('rekognition')
    response = client.detect_labels(Image={'S3Object':{'Bucket':bucket,'Name':photo}},
        MaxLabels=10)
    print('Detected labels for ' + photo) 
    print()   
    for label in response['Labels']:
        print ("Label: " + label['Name'])
    return len(response['Labels'])


def main():
    photo='cutecat.jpg'
    bucket='bucket-album'
    label_count=detect_labels(photo, bucket)
    print("Labels detected: " + str(label_count))


In [2]:
main()

/home/aditya/.pyenv/versions/cloud/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


Detected labels for cutecat.jpg

Label: Couch
Confidence: 96.70336151123047
Instances:
Parents:
   Furniture
----------

Label: Cat
Confidence: 94.3367919921875
Instances:
  Bounding box
    Top: 0.0590653270483017
    Left: 0.24809980392456055
    Width: 0.5100977420806885
    Height: 0.909473180770874
  Confidence: 94.0555648803711

Parents:
   Animal
   Mammal
   Pet
----------

Label: Pet
Confidence: 94.3367919921875
Instances:
Parents:
   Animal
----------

Label: Animal
Confidence: 94.3367919921875
Instances:
Parents:
----------

Label: Mammal
Confidence: 94.3367919921875
Instances:
Parents:
   Animal
----------

Label: Kitten
Confidence: 93.94374084472656
Instances:
Parents:
   Animal
   Cat
   Mammal
   Pet
----------

Label: Cushion
Confidence: 57.85679626464844
Instances:
Parents:
   Home Decor
----------

Label: Manx
Confidence: 56.86974334716797
Instances:
Parents:
   Animal
   Cat
   Mammal
   Pet
----------

Label: Abyssinian
Confidence: 56.19367218017578
Instances:
Paren

In [7]:
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
import boto3
import pandas as pd

In [9]:
host = 'search-photos-fi6tkdtpua4mm3lpfuebg26sdi.us-east-1.es.amazonaws.com' # cluster endpoint, for example: my-test-domain.us-east-1.es.amazonaws.com
region = 'us-east-1' # e.g. us-west-1

credentials = boto3.Session().get_credentials()
auth = AWSV4SignerAuth(credentials, region)
index_name = 'photo'

In [10]:
client = OpenSearch(
    hosts = [{'host': host, 'port': 443}],
    http_auth = auth,
    use_ssl = True,
    verify_certs = True,
    connection_class = RequestsHttpConnection
)

In [11]:
response = client.indices.create(index_name)
print('\nCreating index:')
print(response)


Creating index:
{'acknowledged': True, 'shards_acknowledged': True, 'index': 'photo'}


In [16]:
opensearch_id = datetime.now().strftime("%Y%m%d%H%M%S%f")
document = {
        'label': label,
        's3_path': "https://{}.s3.amazonaws.com/{}".format(bucket, key)
    }
response = client.index(
    index = index_name,
    body = document,
    id = i,
    refresh = True
)

'20221106202719569851'

In [32]:
import requests

In [33]:
bucket =  'bucket-album'
key = 'hello.jpg'
label = 'cuki'

In [36]:
host = 'https://search-photos-fi6tkdtpua4mm3lpfuebg26sdi.us-east-1.es.amazonaws.com' # the OpenSearch Service domain, e.g. https://search-mydomain.us-west-1.es.amazonaws.com
index = 'photo'
type = '_doc'
url = host + '/' + index + '/' + type
headers = { "Content-Type": "application/json" }

In [42]:
document = {
        'label': label,
        's3_path': "https://{}.s3.amazonaws.com/{}".format(bucket, key)
    }
r = requests.post(url, auth=auth, json=document, headers=headers)

In [43]:
r

<Response [201]>

In [2]:
a = [[1,2,3,4],[2,3,4],[3,4,5],[3,4,5,6]]
set.intersection(*map(set,a))

{3, 4}

In [6]:
set([1,2]).union(set([2,3]))

{1, 2, 3}

In [10]:
"aads".endswith("s")

True